In [1]:
import sys
sys.version

'2.7.11 |Anaconda 2.4.1 (x86_64)| (default, Dec  6 2015, 18:57:58) \n[GCC 4.2.1 (Apple Inc. build 5577)]'

In [2]:
import math
import time

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *

<div class="alert alert-info">
<strong>LOAD DATA</strong>
</div>

In [4]:
start = time.time()

"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

timer = time.time() - start
print '%d seconds to load data' % timer

67 seconds to load data


In [5]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [6]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


<div class="alert alert-info">
<strong>CREATION OF VALIDATION SET FROM TRAINING DATA</strong>
</div>

Given the abundance of training data, we have chosen to use a validation set (or "test set" within our training set) to help us compare some of the different models we construct before trying them on the Kaggle set.  We use a common 80-20 split for this purpose. (Reference: http://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio)

In [5]:
# construct validation set (80-20)
cutoff = df_train.shape[0]*4/5
print "Total # of observations: ", df_train.shape[0]
print "# for use in training set: ", cutoff

Total # of observations:  1000000
# for use in training set:  800000


In [6]:
#store gap values
#Y_train = df_train.gap.values
Y_all = df_train.gap.values
Y_train = Y_all[:cutoff]
Y_valid = Y_all[cutoff:]

#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column: we should consider saving this if there's a chance the numbers were not ordered
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [7]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
# save smiles column
smiles = df_all['smiles']
smiles.shape

(1824230,)

In [9]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values  # converts data to numpy format (array)

#X_train = vals[:test_idx]
X_train = vals[:cutoff]
X_valid = vals[cutoff:test_idx]
X_test = vals[test_idx:]

print "Train features: ", X_train.shape
print "Validation set: ", X_valid.shape
print "Train gap: ", Y_train.shape
print "Validation set for gap: ", Y_valid.shape
print "Test features:", X_test.shape

Train features:  (800000, 256)
Validation set:  (200000, 256)
Train gap:  (800000,)
Validation set for gap:  (200000,)
Test features: (824230, 256)


In [12]:
vals.shape

(1824230, 256)

In [13]:
test_idx

1000000

In [10]:
X_all = vals[:test_idx]

<div class="alert alert-info">
<strong>BASELINE PREDICTORS</strong>
</div>

## linear regression

In [15]:
start = time.time()

LR = LinearRegression()
#LR.fit(X_train, Y_train)
LR.fit(X_all, Y_all) # original
LR_pred = LR.predict(X_test)

timer = time.time() - start
print '\n%d seconds to run baseline linear regression' % timer


25 seconds to run baseline linear regression


In [16]:
#LR_R2 = r2_score(Y_train, LR.predict(X_train))
LR_R2 = r2_score(Y_all, LR.predict(X_all))
print 'R_squared = %0.5f' % LR_R2

R_squared = 0.46103


In [17]:
#Source: http://sourceforge.net/p/scikit-learn/mailman/scikit-learn-general/thread/516EC479.6080704@gmail.com/
#LR_adjR2 = 1 - float(len(Y_train)-1)/(len(Y_train)-len(LR.coef_)-1)*(1 - r2_score(Y_train, LR.predict(X_train)))
LR_adjR2 = 1 - float(len(Y_all)-1)/(len(Y_all)-len(LR.coef_)-1)*(1 - r2_score(Y_all, LR.predict(X_all)))
print 'Adjusted R_squared = %0.5f' % LR_adjR2

Adjusted R_squared = 0.46089


In [18]:
#LR_rmse = math.sqrt(mean_squared_error(Y_train, LR.predict(X_train)))
LR_rmse = math.sqrt(mean_squared_error(Y_all, LR.predict(X_all)))
print 'Training set RMSE = %0.5f' % LR_rmse
# print 'Test set RMSE (from leaderboard) = 0.29846'

Training set RMSE = 0.29893


In [19]:
# just to summarize the data in the prediction vector to get a ballpark figure of values
print 'mean of predictions = %0.4f' % np.mean(LR_pred)
print 'median of predictions = %0.4f' % np.median(LR_pred)
print 'minimum of predictions = %0.4f' % np.min(LR_pred)
print 'maximum of predictions = %0.4f' % np.max(LR_pred)

mean of predictions = 1.9159
median of predictions = 1.9624
minimum of predictions = 0.7266
maximum of predictions = 2.6076


In [20]:
LR_pred[0:10]

array([ 1.61085441,  1.64186093,  1.53968802,  1.77994206,  2.28964563,
        2.12677565,  2.19776186,  1.70927539,  2.19615967,  2.07280525])

## random forest

In [21]:
start = time.time()

RF = RandomForestRegressor()
#RF.fit(X_train, Y_train)
RF.fit(X_all, Y_all)
RF_pred = RF.predict(X_test)

timer = time.time() - start
print '\n%d seconds to run baseline random forest' % timer


42 seconds to run baseline random forest


In [22]:
#RF_R2 = r2_score(Y_train, RF.predict(X_train))
RF_R2 = r2_score(Y_all, RF.predict(X_all))
print 'R_squared = %0.5f' % RF_R2

R_squared = 0.55416


In [23]:
#RF_rmse = math.sqrt(mean_squared_error(Y_train, RF.predict(X_train)))
RF_rmse = math.sqrt(mean_squared_error(Y_all, RF.predict(X_all)))
print 'Training set RMSE = %0.5f' % RF_rmse
# print 'Test set RMSE (from leaderboard) = 0.27207'

Training set RMSE = 0.27188


<div class="alert alert-info">
<strong>SAVE RESULTS</strong>
</div>

In [11]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

<div class="alert alert-info">
<strong>TRY: models on DF with reduced features (28) but with interactions</strong>
</div>

Based on our exploratory data analysis, we reduced the dimension of our data set by keeping only the features obtained from a backward selection process (see analysis in R), as a starting point. Essentially, primarily vectors with no variability in the training set were removed.

In [12]:
keep = ["feat_001", "feat_005", "feat_006", "feat_007", "feat_025", "feat_037", "feat_044", "feat_068", "feat_069", 
        "feat_072", "feat_087", "feat_090", "feat_102", "feat_119", "feat_123", "feat_126", "feat_132", "feat_173",
        "feat_176", "feat_187", "feat_196", "feat_199", "feat_208", "feat_225", "feat_226", "feat_248", "feat_251", 
        "feat_252"]

reduced_DF = df_all[keep]

In [13]:
vals2 = reduced_DF.values  # converts data to numpy format (array)
vals2.shape

(1824230, 28)

In [14]:
for i in range(vals2.shape[1]):
    for j in range(i+1, vals2.shape[1]):
        colname = keep[i] + "x" + keep[j][-3:]
        reduced_DF[colname] = reduced_DF[keep[i]]*reduced_DF[keep[j]]

vals2 = reduced_DF.values
vals2.shape

/Users/K-Lo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1824230, 406)

In [27]:
reduced_DF.head()

,feat_001,feat_005,feat_006,feat_007,feat_025,feat_037,feat_044,feat_068,feat_069,feat_072,...,feat_225x226,feat_225x248,feat_225x251,feat_225x252,feat_226x248,feat_226x251,feat_226x252,feat_248x251,feat_248x252,feat_251x252
0,0,1,0,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,1,0,1,1,1,0,0,1,1,...,0,0,0,0,1,1,0,1,0,0
2,1,1,1,1,0,1,0,1,1,1,...,0,0,0,0,1,0,1,0,1,0
3,1,1,1,1,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
4,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [14]:
X_train = vals2[:cutoff]
X_valid = vals2[cutoff:test_idx]
X_test = vals2[test_idx:]

print "Train features: ", X_train.shape
print "Validation set: ", X_valid.shape
print "Train gap: ", Y_train.shape
print "Validation set for gap: ", Y_valid.shape
print "Test features:", X_test.shape

Train features:  (800000, 406)
Validation set:  (200000, 406)
Train gap:  (800000,)
Validation set for gap:  (200000,)
Test features: (824230, 406)


### Linear Regression on set with interactions

In [29]:
start = time.time()

LR2 = LinearRegression()
LR2.fit(X_train, Y_train)
LR2_validate = LR2.predict(X_valid)
# LR2_pred = LR2.predict(X_test)  

timer = time.time() - start
print '\n%d seconds to run linear regression (reduced + interactions)' % timer


56 seconds to run linear regression (reduced + interactions)


In [30]:
LR2_rmse = math.sqrt(mean_squared_error(Y_train, LR2.predict(X_train)))
print 'Training set (80-20) RMSE = %0.5f' % LR2_rmse

LR2_rmsev = math.sqrt(mean_squared_error(Y_valid, LR2_validate))
print 'Validation set RMSE = %0.5f' % LR2_rmsev

Training set (80-20) RMSE = 0.27985
Validation set RMSE = 0.28070


### Random Forest (on set with interactions)

In [31]:
start = time.time()

RF2 = RandomForestRegressor()
RF2.fit(X_train, Y_train)
RF2_validate = RF2.predict(X_valid)
#RF2_pred = RF2.predict(X_test)

timer = time.time() - start
print '\n%d seconds to run random forest (reduced + interactions)' % timer


276 seconds to run random forest (reduced + interactions)


In [32]:
RF2_rmse = math.sqrt(mean_squared_error(Y_train, RF2.predict(X_train)))
print 'Training set (80-20) RMSE = %0.5f' % RF2_rmse

RF2_rmsev = math.sqrt(mean_squared_error(Y_valid, RF2_validate))
print 'Validation set RMSE = %0.5f' % RF2_rmsev

Training set (80-20) RMSE = 0.27161
Validation set RMSE = 0.27337


In [ ]:
# write_to_file("RF2.csv", RF2_pred)
# (RF_pred==RF2_pred).all()

### Lasso 

Link on regularation methods: http://www.datarobot.com/blog/regularized-linear-regression-with-scikit-learn/ <p>

Regularization techniques

In the above example we used Ridge Regression, a regularized linear regression technique that puts an L2 norm penalty on the regression coefficients. Another popular regularization technique is the LASSO, a technique which puts an L1 norm penalty instead. The difference between the two is that the LASSO leads to sparse solutions, driving most coefficients to zero, whereas Ridge Regression leads to dense solutions, in which most coefficients are non-zero...

Another popular regularization technique is the Elastic Net, the convex combination of the L2 norm and the L1 norm. It too leads to a sparse solution.

In [33]:
from sklearn.linear_model import Lasso

In [34]:
start = time.time()

params = [1.0, 0.1, 0.01, 0.001, 0.0001]

for alpha in params:
    LR_lasso = Lasso(alpha)
    LR_lasso.fit(X_train, Y_train)
    LR_lasso_validate = LR_lasso.predict(X_valid)
    # LR_lasso_pred = LR_lasso.predict(X_test)

    timer = time.time() - start
    print '\nAlpha = %0.4f' % alpha
    print '%d seconds to complete lasso on set with interactions' % timer

    LR_lasso_rmse = math.sqrt(mean_squared_error(Y_train, LR_lasso.predict(X_train)))
    print 'Training set (80/20) RMSE = %0.5f' % LR_lasso_rmse

    LR_lasso_rmsev = math.sqrt(mean_squared_error(Y_valid, LR_lasso_validate))
    print 'Validation set RMSE = %0.5f' % LR_lasso_rmsev


Alpha = 1.0000
8 seconds to complete lasso on set with interactions
Training set (80/20) RMSE = 0.40710
Validation set RMSE = 0.40750

Alpha = 0.1000
20 seconds to complete lasso on set with interactions
Training set (80/20) RMSE = 0.40710
Validation set RMSE = 0.40750

Alpha = 0.0100
67 seconds to complete lasso on set with interactions
Training set (80/20) RMSE = 0.30917
Validation set RMSE = 0.30963

Alpha = 0.0010
576 seconds to complete lasso on set with interactions
Training set (80/20) RMSE = 0.28814
Validation set RMSE = 0.28878

Alpha = 0.0001
1432 seconds to complete lasso on set with interactions
Training set (80/20) RMSE = 0.28092
Validation set RMSE = 0.28165


/Users/K-Lo/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


### Ridge Regression

In [35]:
from sklearn.linear_model import Ridge

In [36]:
start = time.time()

params = [0.0, 1e-8, 1e-5, 1e-1]

for alpha in params:
    LR_ridge = Ridge(alpha)
    LR_ridge.fit(X_train, Y_train)
    LR_ridge_validate = LR_ridge.predict(X_valid)
    # LR_ridge_pred = LR_ridge.predict(X_test)

    timer = time.time() - start
    print '\nAlpha = %0.9f' % alpha
    print '%d seconds to run ridge regression on set with interactions' % timer

    LR_ridge_rmse = math.sqrt(mean_squared_error(Y_train, LR_ridge.predict(X_train)))
    print 'Training set (80/20) RMSE = %0.5f' % LR_ridge_rmse

    LR_ridge_rmsev = math.sqrt(mean_squared_error(Y_valid, LR_ridge_validate))
    print 'Validation set RMSE = %0.5f' % LR_ridge_rmsev


Alpha = 0.000000000
91 seconds to run ridge regression on set with interactions
Training set (80/20) RMSE = 0.27989
Validation set RMSE = 0.28074

Alpha = 0.000000010
102 seconds to run ridge regression on set with interactions
Training set (80/20) RMSE = 0.27984
Validation set RMSE = 0.28070

Alpha = 0.000010000
115 seconds to run ridge regression on set with interactions
Training set (80/20) RMSE = 0.27984
Validation set RMSE = 0.28070

Alpha = 0.100000000
126 seconds to run ridge regression on set with interactions
Training set (80/20) RMSE = 0.27984
Validation set RMSE = 0.28070


### Elastic Net

In [37]:
from sklearn.linear_model import ElasticNet

In [38]:
start = time.time()

params = [1.0, 0.1, 0.01, 0.001, 0.0001]

for alpha in params:
    LR_en = ElasticNet(alpha)
    LR_en.fit(X_train, Y_train)
    LR_en_validate = LR_en.predict(X_valid)
    # LR_en_pred = LR_en.predict(X_test)

    timer = time.time() - start
    print '\nAlpha = %0.4f' % alpha
    print '%d seconds to complete Elastic Net regularization on set with interactions' % timer

    LR_en_rmse = math.sqrt(mean_squared_error(Y_train, LR_en.predict(X_train)))
    print 'Training set (80/20) RMSE = %0.5f' % LR_en_rmse

    LR_en_rmsev = math.sqrt(mean_squared_error(Y_valid, LR_en_validate))
    print 'Validation set RMSE = %0.5f' % LR_en_rmsev


Alpha = 1.0000
8 seconds to complete Elastic Net regularization on set with interactions
Training set (80/20) RMSE = 0.40710
Validation set RMSE = 0.40750

Alpha = 0.1000
38 seconds to complete Elastic Net regularization on set with interactions
Training set (80/20) RMSE = 0.36993
Validation set RMSE = 0.37035

Alpha = 0.0100
311 seconds to complete Elastic Net regularization on set with interactions
Training set (80/20) RMSE = 0.30129
Validation set RMSE = 0.30182

Alpha = 0.0010
985 seconds to complete Elastic Net regularization on set with interactions
Training set (80/20) RMSE = 0.28503
Validation set RMSE = 0.28566

Alpha = 0.0001
1982 seconds to complete Elastic Net regularization on set with interactions
Training set (80/20) RMSE = 0.28039
Validation set RMSE = 0.28116


### Let's try transforming the response

See the discussion in the R notebook. This is primarily to deal with heteroskedasticity, which can be a problem for the Linear Regression models.

In [39]:
# Linear Regression
start = time.time()

LR3 = LinearRegression()
LR3.fit(X_train, np.log(Y_train + 1.5))
LR3_validate = LR3.predict(X_valid)
#LR3_pred = LR3.predict(X_test)

timer = time.time() - start
print '\n%d seconds to run linear regression (interactions + transform)' % timer

LR3_rmse = math.sqrt(mean_squared_error(Y_train, np.exp(LR3.predict(X_train))-1.5))
print 'Training set (80/20) RMSE = %0.5f' % LR3_rmse

LR3_rmsev = math.sqrt(mean_squared_error(Y_valid, np.exp(LR3_validate)-1.5))
print 'Validation set RMSE = %0.5f' % LR3_rmsev


61 seconds to run linear regression (interactions + transform)
Training set (80/20) RMSE = 0.27994
Validation set RMSE = 0.28074


In [40]:
# Random Forest
start = time.time()

RF3 = RandomForestRegressor()
RF3.fit(X_train, np.log(Y_train + 1.5))
RF3_validate = RF3.predict(X_valid)
#RF3_pred = RF3.predict(X_test)

timer = time.time() - start
print '\n%d seconds to run random forest (interactions + transform)' % timer

RF3_rmse = math.sqrt(mean_squared_error(Y_train, np.exp(RF3.predict(X_train))-1.5))
print 'Training set (80/20) RMSE = %0.5f' % RF3_rmse

RF3_rmsev = math.sqrt(mean_squared_error(Y_valid, np.exp(RF3_validate)-1.5))
print 'Validation set RMSE = %0.5f' % RF3_rmsev


284 seconds to run random forest (interactions + transform)
Training set (80/20) RMSE = 0.27187
Validation set RMSE = 0.27359


In [ ]:
# just to summarize the data in the prediction vector
# print 'mean of predictions = %0.4f' % np.mean(cnvt_pred)
# print 'median of predictions = %0.4f' % np.median(cnvt_pred)
# print 'minimum of predictions = %0.4f' % np.min(cnvt_pred)
# print 'maximum of predictions = %0.4f' % np.max(cnvt_pred)

In [ ]:
# write_to_file("RF3.csv", cnvt_pred)

So far, the five submissions that we put through beat the Linear Regression baseline but not the Random Forest baseline. Given the evidence of heteroskedasticity, and the fact that all of these models seem to perform at the same level on the test set, all this is generally indicative that we are missing some key features that explain the data better.  We should now explore coming up with new features to consider for further modeling.

<div class="alert alert-info">
<strong>Increasing number of fingerprints</strong>
</div>

After inspecting the documentation for RDKit, we realized that only a subset of "fingerprints" were provided. We tried working with the full set of 2048 **but on training data with fewer (about 50k) observations).**  We removed features where there was no variation in the training set. We ran a number of the models above. 

#### load test data from big set

In [ ]:
#start = time.time()

"""
Read in test data as Pandas DataFrames
"""

#df_test_big = pd.read_csv("test_2048.csv")

#timer = time.time() - start
#print '%d seconds to load test data' % timer

#df_test_big.head()

In [ ]:
#df_test_big.shape

In [ ]:
##clean test file until shape is (824230, 2048) <<<---

#delete first column
#df_test_big = df_test_big.drop(['Unnamed: 0'], axis=1)

#save and delete smiles
#big_smiles_test = df_test_big['smiles']
#df_test_big = df_test_big.drop(['smiles'], axis=1)

#df_test_big.head()

In [ ]:
#X_big_test = df_test_big.values  # converts data to numpy format (array)

In [ ]:
#X_big_test.shape ## shape should be (824230, 2048) <<<---

#### load training data from big set

In [41]:
start = time.time()

"""
Read in train data as Pandas DataFrames
"""
df_train_big = pd.read_csv("train_2048_50k.csv")
# df_test_big = pd.read_csv("test_2048.csv")

timer = time.time() - start
print '%d seconds to load data' % timer

13 seconds to load data


In [42]:
df_train_big.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2040,2041,2042,2043,2044,2045,2046,2047,smiles,gap
0,0,1,1,0,0,0,1,0,0,0,...,1,1,0,1,1,1,1,1,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,1.19
1,1,1,0,1,0,1,1,1,0,0,...,1,1,0,0,1,0,1,0,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.60
2,2,1,1,1,1,1,1,1,0,0,...,0,1,1,1,1,1,1,1,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.49
3,3,1,1,0,0,1,1,1,0,0,...,1,1,0,1,1,0,1,0,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.36
4,4,1,1,1,0,0,1,0,1,0,...,0,1,0,1,0,1,1,1,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,1.98


In [43]:
df_train_big.shape

(47837, 2051)

In [44]:
colsums = df_train_big.sum(axis=0)

In [45]:
min(colsums)

1593

In [46]:
# construct validation set (80-20)
cutoff2 = df_train_big.shape[0]*4/5
print "Total # of observations: ", df_train_big.shape[0]
print "# for use in training set: ", cutoff2

Total # of observations:  47837
# for use in training set:  38269


In [47]:
#store gap values
Y_big_all = df_train_big.gap.values
Y_big_train = Y_big_all[:cutoff2]
Y_big_valid = Y_big_all[cutoff2:]

#delete 'gap' column
df_train_big = df_train_big.drop(['gap'], axis=1)
#delete first column
df_train_big = df_train_big.drop(['Unnamed: 0'], axis=1)

#save and delete smiles
big_smiles = df_train_big['smiles']
df_train_big = df_train_big.drop(['smiles'], axis=1)

df_train_big.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1,1,0,0,0,1,0,0,0,0,...,1,0,1,1,0,1,1,1,1,1
1,1,0,1,0,1,1,1,0,0,0,...,0,1,1,1,0,0,1,0,1,0
2,1,1,1,1,1,1,1,0,0,0,...,1,1,0,1,1,1,1,1,1,1
3,1,1,0,0,1,1,1,0,0,0,...,1,0,1,1,0,1,1,0,1,0
4,1,1,1,0,0,1,0,1,0,0,...,1,1,0,1,0,1,0,1,1,1


In [48]:
vals2 = df_train_big.values  # converts data to numpy format (array)
X_big_train = vals2[:cutoff2]
X_big_valid = vals2[cutoff2:]

print "Train features: ", X_big_train.shape
print "Validation set: ", X_big_valid.shape
print "Train gap: ", Y_big_train.shape
print "Validation set for gap: ", Y_big_valid.shape

Train features:  (38269, 2048)
Validation set:  (9568, 2048)
Train gap:  (38269,)
Validation set for gap:  (9568,)


### models

In [49]:
# Linear Regression
start = time.time()

LRbig = LinearRegression()
LRbig.fit(X_big_train, Y_big_train)
LRbig_validate = LRbig.predict(X_big_valid)

timer = time.time() - start
print '\n%d seconds to run linear regression (2048 fingerprints)' % timer

LRbig_rmse = math.sqrt(mean_squared_error(Y_big_train, LRbig.predict(X_big_train)))
print 'Training set (80/20) RMSE = %0.5f' % LRbig_rmse

LRbig_rmsev = math.sqrt(mean_squared_error(Y_big_valid, LRbig_validate))
print 'Validation set RMSE = %0.5f' % LRbig_rmsev


89 seconds to run linear regression (2048 fingerprints)
Training set (80/20) RMSE = 0.13556
Validation set RMSE = 0.14358


<div class="alert alert-warning">
<strong>Obtain test file for linear regression</strong>
</div>

In [ ]:
#LRbig_pred = LRbig.predict(X_big_test)
#write_to_file("LRbig.csv", LRbig_pred)

<div class="alert alert-warning">
<strong>--</strong>
</div>

In [50]:
# Random Forest
start = time.time()

RFbig = RandomForestRegressor()
RFbig.fit(X_big_train, Y_big_train)
RFbig_validate = RFbig.predict(X_big_valid)

timer = time.time() - start
print '\n%d seconds to run random forest (2048 fingerprints)' % timer

RFbig_rmse = math.sqrt(mean_squared_error(Y_big_train, RFbig.predict(X_big_train)))
print 'Training set (80/20) RMSE = %0.5f' % RFbig_rmse

RFbig_rmsev = math.sqrt(mean_squared_error(Y_big_valid, RFbig_validate))
print 'Validation set RMSE = %0.5f' % RFbig_rmsev


163 seconds to run random forest (2048 fingerprints)
Training set (80/20) RMSE = 0.07112
Validation set RMSE = 0.16244


<div class="alert alert-warning">
<strong>Obtain test file for random forest</strong>
</div>

In [ ]:
#RFbig_pred = RFbig.predict(X_big_test)
#write_to_file("RFbig.csv", RFbig_pred)

<div class="alert alert-warning">
<strong>--</strong>
</div>

In [51]:
# Lasso

start = time.time()

params = [1.0, 0.1, 0.01, 0.001, 0.0001]

for alpha in params:
    LR_lasso2 = Lasso(alpha)
    LR_lasso2.fit(X_big_train, Y_big_train)
    LR_lasso2_validate = LR_lasso2.predict(X_big_valid)
    # LR_lasso2_pred = LR_lasso2.predict(X_big_test)

    timer = time.time() - start
    print '\nAlpha = %0.4f' % alpha
    print '%d seconds to complete lasso on big set (2048 fingerprints)' % timer

    LR_lasso2_rmse = math.sqrt(mean_squared_error(Y_big_train, LR_lasso2.predict(X_big_train)))
    print 'Training set (80/20) RMSE = %0.5f' % LR_lasso2_rmse

    LR_lasso2_rmsev = math.sqrt(mean_squared_error(Y_big_valid, LR_lasso2_validate))
    print 'Validation set RMSE = %0.5f' % LR_lasso2_rmsev


Alpha = 1.0000
2 seconds to complete lasso on big set (2048 fingerprints)
Training set (80/20) RMSE = 0.39679
Validation set RMSE = 0.39483

Alpha = 0.1000
5 seconds to complete lasso on big set (2048 fingerprints)
Training set (80/20) RMSE = 0.39679
Validation set RMSE = 0.39483

Alpha = 0.0100
14 seconds to complete lasso on big set (2048 fingerprints)
Training set (80/20) RMSE = 0.20788
Validation set RMSE = 0.20784

Alpha = 0.0010
97 seconds to complete lasso on big set (2048 fingerprints)
Training set (80/20) RMSE = 0.15339
Validation set RMSE = 0.15558

Alpha = 0.0001
228 seconds to complete lasso on big set (2048 fingerprints)
Training set (80/20) RMSE = 0.13709
Validation set RMSE = 0.14350


<div class="alert alert-info">
<strong>Try string parsing</strong>
</div>

In [52]:
smiles[1]

1    C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...
1           [nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1
Name: smiles, dtype: object

We notice the string itself is composed of common characters. Among other things, chemical symbols and bonds are represented.  Perhaps if we treat this as a string character investigative exercise we may be able to generate some relevant features: e.g. whether or not a particular character(s) appears, and if so, how many times. <p>

Link re SMILES strings:  https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system <p>

Some points: <br>
- a number of equally valid SMILES strings can be written for a molecule. For example, CCO, OCC and C(O)C all specify the structure of ethanol <br>
- Atoms are represented by the standard abbreviation of the chemical elements, in square brackets, such as [Au] for gold. Brackets can be omitted for the "organic subset" of B, C, N, O, P, S, F, Cl, Br, and I. All other elements must be enclosed in brackets. If the brackets are omitted, the proper number of implicit hydrogen atoms is assumed; for instance the SMILES for water is simply O <br>
- An atom holding one or more electrical charges is enclosed in brackets, followed by the symbol H if it is bonded to one or more atoms of hydrogen, followed by the number of hydrogen atoms <br>
- Ring closure labels are used to indicate connectivity between non-adjacent atoms in the SMILES string, which for cyclohexane and dioxane can be written as C1CCCCC1 and O1CCOCC1 respectively. For a second ring, the label will be 2 (naphthalene: c1cccc2c1cccc2 (note the lower case for aromatic compounds))... After reaching 9, the label must be preceded by a '%' <br>
- Double, triple, and quadruple bonds are represented by the symbols '=', '#', and '$' respectively as illustrated by the SMILES O=C=O (carbon dioxide) <br>
- Aromatic C, O, S and N atoms are shown in their lower case 'c', 'o', 's' and 'n' respectively. Benzene, pyridine and furan can be represented respectively by the SMILES c1ccccc1, n1ccccc1 and o1cccc1 <br>
- Aromatic nitrogen bonded to hydrogen, as found in pyrrole must be represented as [nH] and imidazole is written in SMILES notation as n1c[nH]cc1 <br>
Branches are described with parentheses <br>
- Configuration around double bonds is specified using the characters "/" and "\" <br>
- Configuration at tetrahedral carbon is specified by @ or @@ <p>

Link re: Organic Solar cells: https://en.wikipedia.org/wiki/Organic_solar_cell <p>

- Molecular engineering (e.g. changing the length and functional group of polymers) can change the band gap, allowing for electronic tunability <br>
- A common characteristic of both the small molecules and polymers (Fig 1) used as the light-absorbing material in photovoltaics is that they all have large conjugated systems. A conjugated system is formed where carbon atoms covalently bond with alternating single and double bonds

Searching through the original training data set revealed that the following characters were present in the string: <p>

numbers = [1, 2, 3, 4, 5, 6] <br>
bond_symbols = ["=", "-"] <br>
carbons = ["c", "C"] <br>
hydrogens = ["H"] (not sure if He appears)<br>
sulphurs = ["S", "s"] (should treat Si separately, not sure if Se/Sn appears) <br>
oxygens = ["o"] <br>
nitrogens = ["n"] <br>
brackets = ["(", ")", "[", "]"] <p>

In [15]:
def countsymb(symbol, smile):
    # count number of times symbol exists in smile string
    # this is specific to the list of characters found in the training set noted above
    # since we check for specific two-symbol elements
    numchars = len(str(symbol))
    count = 0
    
    if numchars>1:
        for i in range(len(smile)+1-numchars):
            if smile[i:i+numchars]==symbol:
                count += 1
    else:
        for i in range(len(smile)):
            if smile[i]==symbol:
                if (((symbol=="S") or (symbol=="s")) and (i!=len(smile)) and (smile[i+1] in ["i", "e", "n"])):
                    pass
                elif (symbol=="H") and (i!=len(smile)) and (smile[i+1]=="e"):
                    pass
                else:
                    try:
                        if (symbol=="H") and (smile[i+2]=="]"):
                            # the number represents hydrogen atoms
                            count += int(smile[i+1])
                        else:
                            count += 1
                    except:
                        count += 1

    return count

In [54]:
countsymb("H", "CcH3]eCcK")

3

In [55]:
"i" not in ["i", "j"]

False

In [56]:
str(smiles.loc[0])

'0    c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...\n0    c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...\nName: smiles, dtype: object'

In [57]:
countsymb("Si", str(smiles.loc[1]))

2

In [16]:
def maxnum(smile):
    # determines the maximum integer that occurs in a smile string as part of a ring
    maxn = 0
    for i in range(len(smile)):
        if smile[i]=="%":  # 2 digit
            if int(smile[i+1:i+3])>maxn:
                maxn = int(smile[i+1:i+3])
        else:
            try:
                if int(smile[i])>maxn:
                    if (i!=len(smile)) and (smile[i+1]=="]"):
                        # not part of ring, it is a hydrogen count
                        pass
                    else:
                        maxn = int(smile[i])
            except:
                pass
    return maxn

In [59]:
maxnum("Csslkcjlkjd13lkjflkjf2kjhkjhkjh7")

3

In [60]:
maxnum(str(smiles.loc[1]))

4

### generate features only from string

Given that the 256 features we were given did not show any significant improvement, we decided to start from scratch.

In [17]:
smileparseDF = smiles.to_frame()

In [62]:
smileparseDF.shape

(1824230, 1)

In [18]:
# calculate some feature for each smile string and add a feature column

# count carbons
smallc = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("c", x)))
smileparseDF['smallc'] = pd.DataFrame(smallc)

bigc = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("C", x)))
smileparseDF['bigc'] = pd.DataFrame(bigc)

smileparseDF['sumc'] = smileparseDF['smallc'] + smileparseDF['bigc']

# count H, o, n
bigh = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("H", x)))
smileparseDF['bigh'] = pd.DataFrame(bigh)

smallo = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("o", x)))
smileparseDF['smallo'] = pd.DataFrame(smallo)

smalln = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("n", x)))
smileparseDF['smalln'] = pd.DataFrame(smalln)

# count sulphurs
smalls = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("s", x)))
smileparseDF['smalls'] = pd.DataFrame(smalls)

bigs = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("S", x)))
smileparseDF['bigs'] = pd.DataFrame(bigs)

smileparseDF['sums'] = smileparseDF['smalls'] + smileparseDF['bigs']

# two-character
nh = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("nH", x)))
smileparseDF['nh'] = pd.DataFrame(nh)

silica = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("Si", x)))
smileparseDF['silica'] = pd.DataFrame(silica)

sn = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("Sn", x)))
smileparseDF['sn'] = pd.DataFrame(sn)

se = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("se", x)))
smileparseDF['se'] = pd.DataFrame(se)

he = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("He", x)))
smileparseDF['he'] = pd.DataFrame(he)

# carbon sequences
cc = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("cc", x)))
smileparseDF['cc'] = pd.DataFrame(cc)

cc_cap = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("CC", x)))
smileparseDF['cc_cap'] = pd.DataFrame(cc_cap)

cc_bs = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("Cc", x)))
smileparseDF['cc_bs'] = pd.DataFrame(cc_bs)

cc_sb = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("cC", x)))
smileparseDF['cc_sb'] = pd.DataFrame(cc_sb)

smileparseDF['cc_pairs'] = smileparseDF['cc'] + smileparseDF['cc_cap'] + smileparseDF['cc_bs'] + smileparseDF['cc_sb']

cn = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("cn", x)))
smileparseDF['cn'] = pd.DataFrame(cn)

nc = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("nc", x)))
smileparseDF['nc'] = pd.DataFrame(nc)

# bonds
dblbond = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("=", x)))
smileparseDF['dblbond'] = pd.DataFrame(dblbond)

sglbond = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("-", x)))
smileparseDF['sglbond'] = pd.DataFrame(sglbond)

# brackets
roundbr = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("(", x)))
smileparseDF['roundbr'] = pd.DataFrame(roundbr)

squarebr = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("[", x)))
smileparseDF['squarebr'] = pd.DataFrame(squarebr)

smileparseDF['bracks'] = smileparseDF['roundbr'] + smileparseDF['squarebr']

# other compounds
benz = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: countsymb("c1ccccc1", x)))
smileparseDF['benz'] = pd.DataFrame(benz)

# count number of rings
rings = np.vstack(smileparseDF.smiles.astype(str).apply(lambda x: maxnum(x)))
smileparseDF['rings'] = pd.DataFrame(rings)

In [19]:
smileparseDF.head(10)

,smiles,smallc,bigc,sumc,bigh,smallo,smalln,smalls,bigs,sums,...,cc_pairs,cn,nc,dblbond,sglbond,roundbr,squarebr,bracks,benz,rings
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,19,0,19,0,1,3,2,0,2,...,7,1,2,0,3,3,1,4,0,3
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,12,9,21,4,0,2,0,0,0,...,5,0,2,5,0,1,2,3,0,4
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,18,3,21,3,0,3,1,0,1,...,8,0,1,1,3,2,2,4,0,5
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,18,5,23,5,1,1,0,0,0,...,7,0,0,4,2,1,3,4,0,3
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,20,0,20,0,2,6,0,0,0,...,2,3,4,0,1,1,0,1,0,5
5,C1=Cc2cnc3cc4cc(-c5scc6[nH]ccc56)c5ccccc5c4cc3...,23,2,25,3,0,2,1,0,1,...,11,1,1,1,1,1,2,3,0,6
6,c1ncc(s1)-c1cnc2c(c1)oc1c2ccc2ccccc12,18,0,18,0,1,2,1,0,1,...,7,1,2,0,1,2,0,2,0,2
7,c1sc(-c2ccc3c(c2)sc2c3c3=CCC=c3c3cccnc23)c2[se...,21,3,24,0,0,1,2,0,2,...,8,1,1,2,1,2,1,3,0,3
8,c1ccc(o1)-c1cc2cc3cc4c5c[nH]cc5ccc4cc3cc2o1,22,0,22,1,2,1,0,0,0,...,10,0,0,0,1,1,1,2,0,5
9,[nH]1ccc2c3c[nH]cc3c3cc(-c4cncs4)c4=CCC=c4c3c12,17,3,20,2,0,3,1,0,1,...,6,1,1,2,1,1,2,3,0,4


In [65]:
smileparseDF["sn"].sum()

0

In [66]:
smileparseDF["he"].sum()

0

In [21]:
smileparseDF["benz"].sum()

36621

In [20]:
#delete zero columns
smileparseDF = smileparseDF.drop(['sn'], axis=1)
smileparseDF = smileparseDF.drop(['he'], axis=1)

In [21]:
#drop smiles
smileparseDF = smileparseDF.drop(['smiles'], axis=1)

In [25]:
parsevals = smileparseDF.values  # converts data to numpy format (array)

X_train = parsevals[:cutoff]
X_valid = parsevals[cutoff:test_idx]
X_test = parsevals[test_idx:]

print "Train features: ", X_train.shape
print "Validation set: ", X_valid.shape
print "Train gap: ", Y_train.shape
print "Validation set for gap: ", Y_valid.shape
print "Test features:", X_test.shape

Train features:  (800000, 26)
Validation set:  (200000, 26)
Train gap:  (800000,)
Validation set for gap:  (200000,)
Test features: (824230, 26)


In [26]:
# standardize continuous values
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train = stdsc.fit_transform(X_train)
X_valid = stdsc.transform(X_valid)
X_test = stdsc.transform(X_test)

/Users/K-Lo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/K-Lo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/K-Lo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/K-Lo/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### run models

In [27]:
# Linear Regression
start = time.time()

LRparse = LinearRegression()
LRparse.fit(X_train, Y_train)
LRparse_validate = LRparse.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run linear regression (string parse)' % timer

LRparse_rmse = math.sqrt(mean_squared_error(Y_train, LRparse.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % LRparse_rmse

LRparse_rmsev = math.sqrt(mean_squared_error(Y_valid, LRparse_validate))
print 'Validation set RMSE = %0.5f' % LRparse_rmsev


1 seconds to run linear regression (string parse)
Training set (80/20) RMSE = 0.26974
Validation set RMSE = 0.27019


In [28]:
# Random Forest
start = time.time()

RFparse = RandomForestRegressor()
RFparse.fit(X_train, Y_train)
RFparse_validate = RFparse.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run random forest (string parse)' % timer

RFparse_rmse = math.sqrt(mean_squared_error(Y_train, RFparse.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % RFparse_rmse

RFparse_rmsev = math.sqrt(mean_squared_error(Y_valid, RFparse_validate))
print 'Validation set RMSE = %0.5f' % RFparse_rmsev


60 seconds to run random forest (string parse)
Training set (80/20) RMSE = 0.13420
Validation set RMSE = 0.17760


In [30]:
LRparse_pred = LRparse.predict(X_test)
#write_to_file("RFparse.csv", RFparse_pred)

In [ ]:
#RFparse_pred.shape

In [31]:
# just to summarize the data in the prediction vector
print 'mean of predictions = %0.4f' % np.mean(LRparse_pred)
print 'median of predictions = %0.4f' % np.median(LRparse_pred)
print 'minimum of predictions = %0.4f' % np.min(LRparse_pred)
print 'maximum of predictions = %0.4f' % np.max(LRparse_pred)

mean of predictions = 1.9158
median of predictions = 1.9122
minimum of predictions = 0.5389
maximum of predictions = 3.0249


In [87]:
# Lasso
start = time.time()

params = [1.0, 0.1, 0.01, 0.001, 0.0001]

for alpha in params:
    LRparse = Lasso(alpha)
    LRparse.fit(X_train, Y_train)
    LRparse_validate = LRparse.predict(X_valid)
    #LRparse_pred = LR_lasso2.predict(X_test)

    timer = time.time() - start
    print '\nAlpha = %0.4f' % alpha
    print '%d seconds to complete lasso (string parse)' % timer

    LRparse_rmse = math.sqrt(mean_squared_error(Y_train, LRparse.predict(X_train)))
    print 'Training set (80/20) RMSE = %0.5f' % LRparse_rmse

    LRparse_rmsev = math.sqrt(mean_squared_error(Y_valid, LRparse_validate))
    print 'Validation set RMSE = %0.5f' % LRparse_rmsev


Alpha = 1.0000
0 seconds to complete lasso (string parse)
Training set (80/20) RMSE = 0.40710
Validation set RMSE = 0.40750

Alpha = 0.1000
3 seconds to complete lasso (string parse)
Training set (80/20) RMSE = 0.32687
Validation set RMSE = 0.32699

Alpha = 0.0100
11 seconds to complete lasso (string parse)
Training set (80/20) RMSE = 0.25249
Validation set RMSE = 0.25298

Alpha = 0.0010
84 seconds to complete lasso (string parse)
Training set (80/20) RMSE = 0.24348
Validation set RMSE = 0.24394

Alpha = 0.0001
175 seconds to complete lasso (string parse)
Training set (80/20) RMSE = 0.24295
Validation set RMSE = 0.24340


### add additional columns re: bonds

We add 3 additional extracted features generated using RDKit, with a more accurate count of aromatic, single, and double bonds. We also tried adding an interaction of these. We rerun the random forest.

In [22]:
start = time.time()

"""
Read in additional data as Pandas DataFrames
"""
df_add = pd.read_csv("bond_type_counts.csv")

timer = time.time() - start
print '%d seconds to load data' % timer

0 seconds to load data


In [90]:
df_add.head()

,Unnamed: 0,Single,Double,Aromatic
0,0,3,0,29
1,1,9,5,16
2,2,7,1,25
3,3,8,4,21
4,4,1,0,34


In [91]:
df_add.shape

(1824230, 4)

In [92]:
(smileparseDF['dblbond']==df_add['Double']).all()

False

Maybe a different way of counting?

In [93]:
(smileparseDF['sglbond']==df_add['Single']).all()

False

In [23]:
smileparseDF['RDKit_single'] = df_add['Single']
smileparseDF['RDKit_double'] = df_add['Double']
smileparseDF['RDKit_aromatic'] = df_add['Aromatic']

In [34]:
smileparseDF.head()

,smallc,bigc,sumc,bigh,smallo,smalln,smalls,bigs,sums,nh,...,dblbond,sglbond,roundbr,squarebr,bracks,benz,rings,RDKit_single,RDKit_double,RDKit_aromatic
0,19,0,19,0,1,3,2,0,2,0,...,0,3,3,1,4,0,3,3,0,29
1,12,9,21,4,0,2,0,0,0,0,...,5,0,1,2,3,0,4,9,5,16
2,18,3,21,3,0,3,1,0,1,1,...,1,3,2,2,4,0,5,7,1,25
3,18,5,23,5,1,1,0,0,0,1,...,4,2,1,3,4,0,3,8,4,21
4,20,0,20,0,2,6,0,0,0,0,...,0,1,1,0,1,0,5,1,0,34


In [35]:
parsevals = smileparseDF.values  # converts data to numpy format (array)

cutoff = 800000
X_train = parsevals[:cutoff]
X_valid = parsevals[cutoff:test_idx]
X_test = parsevals[test_idx:]

print "Train features: ", X_train.shape
print "Validation set: ", X_valid.shape
print "Train gap: ", Y_train.shape
print "Validation set for gap: ", Y_valid.shape
print "Test features:", X_test.shape

Train features:  (800000, 29)
Validation set:  (200000, 29)
Train gap:  (800000,)
Validation set for gap:  (200000,)
Test features: (824230, 29)


In [ ]:
#stdsc = StandardScaler()
#X_train = stdsc.fit_transform(X_train)
#X_valid = stdsc.transform(X_valid)
#X_test = stdsc.transform(X_test)

In [81]:
# Random Forest
start = time.time()

RFparse2 = RandomForestRegressor()
RFparse2.fit(X_train, Y_train)
RFparse2_validate = RFparse2.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run random forest (string parse + 3 added bond counts)' % timer

RFparse2_rmse = math.sqrt(mean_squared_error(Y_train, RFparse2.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % RFparse2_rmse

RFparse2_rmsev = math.sqrt(mean_squared_error(Y_valid, RFparse2_validate))
print 'Validation set RMSE = %0.5f' % RFparse2_rmsev


65 seconds to run random forest (string parse + 3 added bond counts & interactions)
Training set (80/20) RMSE = 0.11803
Validation set RMSE = 0.15821


In [ ]:
#RFparse2_pred = RFparse2.predict(X_test)
#write_to_file("RFparse2.csv", RFparse2_pred)

In [ ]:
# just to summarize the data in the prediction vector
#print 'mean of predictions = %0.4f' % np.mean(RFparse2_pred)
#print 'median of predictions = %0.4f' % np.median(RFparse2_pred)
#print 'minimum of predictions = %0.4f' % np.min(RFparse2_pred)
#print 'maximum of predictions = %0.4f' % np.max(RFparse2_pred)

### try only three bond features

In [96]:
df_add = df_add.drop(['Unnamed: 0'], axis=1)
df_add.head()

,Single,Double,Aromatic
0,3,0,29
1,9,5,16
2,7,1,25
3,8,4,21
4,1,0,34


In [97]:
parsevals = df_add.values  # converts data to numpy format (array)

X_train = parsevals[:cutoff]
X_valid = parsevals[cutoff:test_idx]
X_test = parsevals[test_idx:]

print "Train features: ", X_train.shape
print "Validation set: ", X_valid.shape
print "Train gap: ", Y_train.shape
print "Validation set for gap: ", Y_valid.shape
print "Test features:", X_test.shape

Train features:  (800000, 3)
Validation set:  (200000, 3)
Train gap:  (800000,)
Validation set for gap:  (200000,)
Test features: (824230, 3)


In [98]:
# Linear Regression
start = time.time()

LRbond = LinearRegression()
LRbond.fit(X_train, Y_train)
LRbond_validate = LRbond.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run linear regression (three bond features only)' % timer

LRbond_rmse = math.sqrt(mean_squared_error(Y_train, LRbond.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % LRbond_rmse

LRbond_rmsev = math.sqrt(mean_squared_error(Y_valid, LRbond_validate))
print 'Validation set RMSE = %0.5f' % LRbond_rmsev


0 seconds to run linear regression (three bond features only)
Training set (80/20) RMSE = 0.32455
Validation set RMSE = 0.32470


In [99]:
# Random Forest
start = time.time()

RFbond = RandomForestRegressor()
RFbond.fit(X_train, Y_train)
RFbond_validate = RFbond.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run random forest (three bond features only)' % timer

RFbond_rmse = math.sqrt(mean_squared_error(Y_train, RFbond.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % RFbond_rmse

RFbond_rmsev = math.sqrt(mean_squared_error(Y_valid, RFbond_validate))
print 'Validation set RMSE = %0.5f' % RFbond_rmsev


6 seconds to run random forest (three bond features only)
Training set (80/20) RMSE = 0.26338
Validation set RMSE = 0.26445


### try combining features (reduced DF with interations plus bond features plus character strings)

In [101]:
smileparseDF.head()

,smallc,bigc,sumc,bigh,smallo,smalln,smalls,bigs,sums,nh,...,nc,dblbond,sglbond,roundbr,squarebr,bracks,rings,RDKit_single,RDKit_double,RDKit_aromatic
0,19,0,19,0,1,3,2,0,2,0,...,2,0,3,3,1,4,3,3,0,29
1,12,9,21,4,0,2,0,0,0,0,...,2,5,0,1,2,3,4,9,5,16
2,18,3,21,3,0,3,1,0,1,1,...,1,1,3,2,2,4,5,7,1,25
3,18,5,23,5,1,1,0,0,0,1,...,0,4,2,1,3,4,3,8,4,21
4,20,0,20,0,2,6,0,0,0,0,...,4,0,1,1,0,1,5,1,0,34


In [102]:
reduced_DF.head()

,feat_001,feat_005,feat_006,feat_007,feat_025,feat_037,feat_044,feat_068,feat_069,feat_072,...,feat_225x226,feat_225x248,feat_225x251,feat_225x252,feat_226x248,feat_226x251,feat_226x252,feat_248x251,feat_248x252,feat_251x252
0,0,1,0,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,1,0,1,1,1,0,0,1,1,...,0,0,0,0,1,1,0,1,0,0
2,1,1,1,1,0,1,0,1,1,1,...,0,0,0,0,1,0,1,0,1,0
3,1,1,1,1,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
4,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [24]:
# join dataframes
combineDF = pd.concat((smileparseDF, reduced_DF), axis=1)
combineDF.shape

(1824230, 435)

In [25]:
parsevalscomb = combineDF.values  # converts data to numpy format (array)

X_train = parsevalscomb[:cutoff]
X_valid = parsevalscomb[cutoff:test_idx]
X_test = parsevalscomb[test_idx:]

print "Train features: ", X_train.shape
print "Validation set: ", X_valid.shape
print "Train gap: ", Y_train.shape
print "Validation set for gap: ", Y_valid.shape
print "Test features:", X_test.shape

Train features:  (800000, 435)
Validation set:  (200000, 435)
Train gap:  (800000,)
Validation set for gap:  (200000,)
Test features: (824230, 435)


In [27]:
# from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train = stdsc.fit_transform(X_train)
X_valid = stdsc.transform(X_valid)
X_test = stdsc.transform(X_test)

In [28]:
# Linear Regression
start = time.time()

LRcomb = LinearRegression()
LRcomb.fit(X_train, Y_train)
LRcomb_validate = LRcomb.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run linear regression (combo)' % timer

LRcomb_rmse = math.sqrt(mean_squared_error(Y_train, LRcomb.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % LRcomb_rmse

LRcomb_rmsev = math.sqrt(mean_squared_error(Y_valid, LRcomb_validate))
print 'Validation set RMSE = %0.5f' % LRcomb_rmsev


51 seconds to run linear regression (combo)
Training set (80/20) RMSE = 0.19460
Validation set RMSE = 0.19536


In [107]:
# Random Forest
start = time.time()

RFcomb = RandomForestRegressor()
RFcomb.fit(X_train, Y_train)
RFcomb_validate = RFcomb.predict(X_valid)

timer = time.time() - start
print '\n%d seconds to run random forest (combo)' % timer

RFcomb_rmse = math.sqrt(mean_squared_error(Y_train, RFcomb.predict(X_train)))
print 'Training set (80/20) RMSE = %0.5f' % RFcomb_rmse

RFcomb_rmsev = math.sqrt(mean_squared_error(Y_valid, RFcomb_validate))
print 'Validation set RMSE = %0.5f' % RFcomb_rmsev


475 seconds to run random forest (combo)
Training set (80/20) RMSE = 0.09788
Validation set RMSE = 0.14944


In [110]:
# RFcomb_pred = RFcomb.predict(X_test)
# write_to_file("RFcomb.csv", RFcomb_pred)

In [111]:
# just to summarize the data in the prediction vector
print 'mean of predictions = %0.4f' % np.mean(RFcomb_pred)
print 'median of predictions = %0.4f' % np.median(RFcomb_pred)
print 'minimum of predictions = %0.4f' % np.min(RFcomb_pred)
print 'maximum of predictions = %0.4f' % np.max(RFcomb_pred)

mean of predictions = 1.9123
median of predictions = 1.9080
minimum of predictions = 0.8150
maximum of predictions = 3.4680
